In [31]:
import pandas as pd
import numpy as np
from scipy.spatial import ConvexHull

In [6]:
set_a = pd.read_csv('./a.csv')
set_b = pd.read_csv('./b.csv')

### Distance function (squared)

In [29]:
def find_dist_squared(x1, y1, x2, y2):
    dist = (x1-x2)**2+(y1-y2)**2
    return(dist)

### Convex hull (for max distances)

In [35]:
def findConvexHull(point_set):
    points = point_set[['X', 'Y']].values
    hull = ConvexHull(points)
    
    #Find the convex hull points
    hull_points = points[hull.vertices]
    
    #Find hull point IDs
    hull_ids = point_set.loc[hull.vertices, 'ID'].tolist()
    
    return hull_points, hull_ids

In [109]:
def furthest_pair(a, b):
    a_hull, a_ids = findConvexHull(a)
    b_hull, b_ids = findConvexHull(b)
    #the max distance points will lie on the convex hull of each point set
    max_dist = -1

    furthest_pair = []

    for i in range(len(a_ids)):
        for j in range(len(b_ids)):
            dist = find_dist_squared(a_hull[i][0], a_hull[i][1], b_hull[j][0], b_hull[j][1])
            if max_dist < dist:
                max_dist = dist
                furthest_pair = [a_ids[i], b_ids[j]]
    return(furthest_pair, max_dist**(1/2))

In [110]:
print(furthest_pair(set_a, set_b))

([872, 720], 139.93957618686142)


### Closest pair
Note that this is not guaranteed to find the closest pairs for any given point sets, but since the sets in this example followed a random distribution and were relatively dense, I scanned within an x-range of closest 10%, which I felt was a good enough number

In [111]:
def closest_pair(a, b):
    points_a = a[['X', 'Y', 'ID']].values.tolist()
    points_b = b[['X', 'Y', 'ID']].values.tolist()
    size = len(points_b)
    
    #sort sets on x axis (WLOG y axis, but auto sorts on x)
    points_a.sort()
    points_b.sort()
    
    min_dist = float('inf')
    closest_pair = None
    
    i, j = 0, 0
    while i < len(points_a) and j < len(points_b):
        p1, p2 = points_a[i], points_b[j]
        current_dist = find_dist_squared(p1[0],p1[1], p2[0],p2[1])
        
        if current_dist < min_dist:
            min_dist = current_dist
            closest_pair = (p1, p2)
        
        if p1[2] < p2[2]:
            i += 1
        else:
            j += 1
        
        #check 10% range around the current points
        for k in range(0-(size//10), (size//10)):
            if 0 <= i + k < len(points_a) and 0 <= j < len(points_b):
                p1_k = points_a[i+k]
                d = find_dist_squared(p1_k[0],p1_k[1], p2[0],p2[1])
                if d < min_dist:
                    min_dist = d
                    closest_pair = (p1_k, p2)
            if 0 <= i < len(points_a) and 0 <= j + k < len(points_b):
                p2_k = points_b[j+k]
                d = find_dist_squared(p1[0],p1[1], p2_k[0],p2_k[1])
                if d < min_dist:
                    min_dist = d
                    closest_pair = (p1, p2_k)
    
    return([closest_pair[0][2], closest_pair[1][2]], min_dist**(1/2))


In [112]:
print(closest_pair(set_a,set_b))

([2203.0, 1613.0], 0.0292231613396485)
